In [1]:
project_name="cnnclf"

In [2]:
import os

In [3]:
%pwd

'f:\\ML_Ops_learning\\Kidney_Disease_Classification_MLflow\\notebooks'

In [4]:
os.chdir("../")

In [5]:
%pwd

'f:\\ML_Ops_learning\\Kidney_Disease_Classification_MLflow'

## config/config.yaml

In [6]:
%%writefile -a config/config.yaml

prepare_base_model:
  root_dir: artifacts/prepare_base_model
  base_model_path: artifacts/prepare_base_model/base_model.h5
  updated_base_model_path: artifacts/prepare_base_model/base_model_updated.h5


Appending to config/config.yaml


# Entity Folder
1. Goto entity folder  src-> textSummarizer -> entity-> __init__.py 
2. Paste below code in __init__.py constructor file

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

Into file

In [9]:
project_name

'cnnclf'

In [10]:
%%writefile -a src/{project_name}/entity/__init__.py

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

Appending to src/cnnclf/entity/__init__.py


# Config folder 
1. Copy get_data_validation_config function 
2. Goto config folder src->wine_quality->config->configuration.py paste under ConfigurationManager class
3. Import DataValidationConfig from wine_quality.entity using this line  
from wine_quality.entity import DataValidationConfig

In [ ]:
from cnnclf.constants import *
from cnnclf.utils import read_yaml, create_directories

In [13]:

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config
    

### write code into the file

In [14]:
project_name

'cnnclf'

In [15]:
%%writefile -a src/{project_name}/config/__init__.py

from cnnclf.entity import PrepareBaseModelConfig
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
            config = self.config.prepare_base_model
            
            create_directories([config.root_dir])

            prepare_base_model_config = PrepareBaseModelConfig(
                root_dir=Path(config.root_dir),
                base_model_path=Path(config.base_model_path),
                updated_base_model_path=Path(config.updated_base_model_path),
                params_image_size=self.params.IMAGE_SIZE,
                params_learning_rate=self.params.LEARNING_RATE,
                params_include_top=self.params.INCLUDE_TOP,
                params_weights=self.params.WEIGHTS,
                params_classes=self.params.CLASSES
            )

            return prepare_base_model_config

Appending to src/cnnclf/config/__init__.py


check the file for error messages

# Component folder
1. Copy entire DataValiadtion class
2. Goto Component folder [ src-> wine_quality -> components - > data_validation.py ]
2. Paste DataValiadtion class in data_validation.py
3. Import DataValidationConfig from entity using this line 

from wine_quality.entity import DataValidationConfig

In [16]:
project_name

'cnnclf'

In [23]:
#%%writefile -a src/{project_name}/components/prepare_base_model.py
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from pathlib import Path

In [24]:
# %%writefile -a src/{project_name}/components/prepare_base_model.py
# from cnnclf.entity import PrepareBaseModelConfig

In [25]:
#%%writefile -a src/{project_name}/components/prepare_base_model.py

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



# Pipeline folder

1. Goto Component folder [ src-> wine_quality -> pipeline - > stage_02_data_validation.py ]

2. Paste below code in stage_02_data_validation.py

In [ ]:
# ''' 
# from wine_quality.logging import logger
# from wine_quality.config.configuration import ConfigurationManager
# from wine_quality.components.data_validation import DataValidation
# class DataValidationTrainingPipeline:
#     def __init__(self):
#         pass
#     def main(self):
        
# '''

3. Copy try block code 
4. Paste in main function in stage_02_data_validation.py

5. Import dataIngestionconfig from config.configuration using this line  
from wine_quality.config.configuration import ConfigurationManager

6. Import data_validation from  components.data_validation using this line  
from wine_quality.components.data_validation import DataValidation

In [26]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-10-28 09:09:00,162] - cnnclf - INFO - __init__ file - LineNum:31   - yaml file: config\config.yaml loaded successfully
[2023-10-28 09:09:00,168] - cnnclf - INFO - __init__ file - LineNum:31   - yaml file: params.yaml loaded successfully
[2023-10-28 09:09:00,171] - cnnclf - INFO - __init__ file - LineNum:50   - created directory at: artifacts
[2023-10-28 09:09:00,175] - cnnclf - INFO - __init__ file - LineNum:50   - created directory at: artifacts/prepare_base_model
[2023-10-28 09:09:01,694] - tensorflow - WARNING - saving_utils file - LineNum:359   - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


c:\Users\win10\.conda\envs\cnncls\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

into file

In [27]:
project_name

'cnnclf'

In [28]:
%%writefile -a src/{project_name}/pipeline/stage_02_prepare_base_model.py
from cnnclf import logger
from cnnclf.config import ConfigurationManager
from cnnclf.components.prepare_base_model import PrepareBaseModel

class PrepareBaseModelTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        prepare_base_model_config = config.get_prepare_base_model_config()
        prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
        prepare_base_model.get_base_model()
        prepare_base_model.update_base_model()

Overwriting src/cnnclf/pipeline/stage_02_prepare_base_model.py


## main file

In [ ]:
# '''
# 1. Goto main.py and paste below code

# from wine_quality.pipeline.stage_02_data_validation import DataValidationTrainingPipeline
# STAGE_NAME = "Data Validation stage"
# try:
#    logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<") 
#    data_validation = DataValidationTrainingPipeline()
#    data_validation.main()
#    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
# except Exception as e:
#         logger.exception(e)
#         raise e

# 2. Run main.py 
# '''

In [29]:
project_name

'cnnclf'

In [30]:
%%writefile -a src/{project_name}/pipeline/stage_02_prepare_base_model.py
STAGE_NAME = "Prepare base model"
try: 
   logger.info(f"*******************")
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
   prepare_base_model = PrepareBaseModelTrainingPipeline()
   prepare_base_model.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
        logger.exception(e)
        raise 

Appending to src/cnnclf/pipeline/stage_02_prepare_base_model.py


In [17]:
%%writefile -a main.py

from cnnclf.pipeline.stage_02_prepare_base_model import PrepareBaseModelTrainingPipeline
STAGE_NAME = "Prepare base model"
try: 
   logger.info(f"*******************")
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
   prepare_base_model = PrepareBaseModelTrainingPipeline()
   prepare_base_model.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
        logger.exception(e)
        raise 

Appending to main.py
